# Ejercicio: Crear un DataFrame Completo con SWAPI

## Objetivo:
El objetivo de este ejercicio es que los alumnos practiquen la obtención de datos de una API REST y su posterior procesamiento para crear un DataFrame en Pandas que contenga información detallada sobre personajes de Star Wars.

## Instrucciones:

1. Obtener los datos:

- Utilizar la API de Star Wars (SWAPI) para obtener datos sobre los personajes (people) de la saga.
- La URL base para obtener la lista de personajes es: https://swapi.dev/api/people/.
- Dado que los datos están paginados, deberán recorrer todas las páginas para obtener la lista completa de personajes.

2. Crear el DataFrame:

    Crear un DataFrame que incluya las siguientes columnas con la información detallada de cada personaje:

    - id: Identificador único del personaje.
    - name: Nombre del personaje.
    - height: Altura del personaje (en centímetros).
    - mass: Peso del personaje (en kilogramos).
    - hair_color: Color de cabello.
    - skin_color: Color de piel.
    - eye_color: Color de ojos.
    - birth_year: Año de nacimiento.
    - gender: Género.
    - homeworld: Nombre del planeta de origen (obtenido mediante una petición adicional a la API).
    - species: Nombre de la especie del personaje (obtenido mediante una petición adicional si el campo species no está vacío).
    - films: Lista de nombres de películas en las que aparece el personaje (obtenido mediante peticiones adicionales).

3. Procesamiento de datos:

    - Cada entrada en el campo homeworld de los personajes es un enlace a la API, desde el cual se debe obtener el nombre del planeta.

    - El campo species contiene una lista de enlaces. Si no está vacío, se debe obtener el nombre de la especie desde la API.

    - El campo films contiene una lista de enlaces a las películas. Se debe realizar una petición por cada enlace para obtener el título de la película.

4. Entrega del ejercicio:

    - Se debe entregar un archivo .ipynb (Jupyter Notebook) con el código que obtenga los datos, procese las respuestas y construya el DataFrame completo.

    - También deben incluir en el notebook una breve descripción de cómo organizaron el proceso para obtener los datos y construir el DataFrame.

Tips:

- Utilicen la librería requests para realizar las peticiones HTTP.
- Tengan en cuenta que la API utiliza paginación, por lo que deberán iterar sobre las páginas hasta obtener todos los datos.
- Los datos relacionados (homeworld, species, films) requieren realizar peticiones adicionales, así que gestionen adecuadamente los tiempos de espera (pueden considerar utilizar la función time.sleep() para evitar sobrecargar la API).
- Si encuentran campos con datos faltantes o errores, documenten cómo lo manejan en su notebook.

In [1]:
import hashlib
import requests
import datetime
import pandas as pd
from IPython.display import HTML

In [11]:
response = requests.get('https://www.swapi.tech/api/', verify=False)  # Cambia la URL si es necesario
print(f"Status Code: {response.status_code}")
print(f"Content: {response.text}")


Status Code: 200
Content: {"message":"ok","result":{"films":"https://www.swapi.tech/api/films","people":"https://www.swapi.tech/api/people","planets":"https://www.swapi.tech/api/planets","species":"https://www.swapi.tech/api/species","starships":"https://www.swapi.tech/api/starships","vehicles":"https://www.swapi.tech/api/vehicles"}}


In [13]:
import requests
import pandas as pd

# Función para obtener datos de todas las páginas
def fetch_all_characters(url):
    characters = []
    while url:
        response = requests.get(url)
        data = response.json()
        characters.extend(data['results'])
        url = data.get('next')  # Avanza a la siguiente página
    return characters

# Función para obtener el nombre del planeta
def get_homeworld_name(homeworld_url):
    response = requests.get(homeworld_url)
    return response.json().get('name', 'Unknown')

# Función para obtener el nombre de las especies
def get_species_name(species_urls):
    species_names = []
    for url in species_urls:
        response = requests.get(url)
        species_names.append(response.json().get('name', 'Unknown'))
    return ', '.join(species_names) if species_names else 'Unknown'

# Función para obtener los nombres de las películas
def get_film_titles(film_urls):
    film_titles = []
    for url in film_urls:
        response = requests.get(url)
        film_titles.append(response.json().get('title', 'Unknown'))
    return ', '.join(film_titles)

# Obtener los datos de los personajes
base_url = "https://www.swapi.tech/api/"
characters = fetch_all_characters(base_url)

# Procesar los datos y construir el DataFrame
data = []
for i, char in enumerate(characters, start=1):
    char_data = {
        'id': i,
        'name': char.get('name', 'Unknown'),
        'height': char.get('height', 'Unknown'),
        'mass': char.get('mass', 'Unknown'),
        'hair_color': char.get('hair_color', 'Unknown'),
        'skin_color': char.get('skin_color', 'Unknown'),
        'eye_color': char.get('eye_color', 'Unknown'),
        'birth_year': char.get('birth_year', 'Unknown'),
        'gender': char.get('gender', 'Unknown'),
        'homeworld': get_homeworld_name(char.get('homeworld', '')),
        'species': get_species_name(char.get('species', [])),
        'films': get_film_titles(char.get('films', []))
    }
    data.append(char_data)

# Crear el DataFrame
df = pd.DataFrame(data)

# Mostrar los primeros registros
print(df.head())

# Guardar en un archivo CSV para inspección
df.to_csv('star_wars_characters.csv', index=False)

KeyError: 'results'

In [2]:
def hash_params(timestamp,priv_key,pub_key):
    """ Marvel API requires server side API calls to include
    md5 hash of timestamp + public key + private key """

    hash_md5 = hashlib.md5()
    hash_md5.update(f'{timestamp}{priv_key}{pub_key}'.encode('utf-8'))
    hashed_params = hash_md5.hexdigest()

    return hashed_params

In [3]:
timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S')

pub_key = '76416694f55964f82f3a51d486d4fc04'
priv_key = '9aca74671cced72ee7efd35e4c621e08cc65c016'

params = {'ts': timestamp, 
        'apikey': pub_key, 
        'hash': hash_params(timestamp,priv_key,pub_key),
        # 'nameStartsWith':"",
        # 'offset':,
        # 'limit': 100,
        'name':'hulk'
        }

url = 'https://gateway.marvel.com/docs/public'

res = requests.get(url,params=params)
res.json()

SSLError: HTTPSConnectionPool(host='gateway.marvel.com', port=443): Max retries exceeded with url: /docs/public?ts=2025-01-0318%3A24%3A07&apikey=76416694f55964f82f3a51d486d4fc04&hash=4a75a9b39b4f16b6f040371a88227cfb&name=hulk (Caused by SSLError(SSLError(1, '[SSL: WRONG_SIGNATURE_TYPE] wrong signature type (_ssl.c:1006)')))

In [4]:
# Función para obtener personajes
def obtener_personajes(url, params):
    res = requests.get(url, params=params)
    res.raise_for_status()
    return res.json()

In [10]:
# Función para obtener los datos adicionales como el planeta, especie y películas
def obtener_datos_adicionales(personaje):
    # Obtener el planeta
    homeworld = requests.get(personaje['homeworld']).json()['name']
    
    # Obtener las especies (si existen)
    especies = []
    if personaje['species']:
        for especie_url in personaje['species']:
            especie = requests.get(especie_url).json()['name']
            especies.append(especie)
    
    # Obtener las películas
    peliculas = []
    for film_url in personaje['films']:
        pelicula = requests.get(film_url).json()['title']
        peliculas.append(pelicula)
    
    return homeworld, especies, peliculas

In [11]:
# Crear un DataFrame de los personajes
def crear_dataframe():
    personajes = obtener_personajes()
    
    data = []
    for p in personajes:
        homeworld, especies, peliculas = obtener_datos_adicionales(p)
        
        # Extraemos los datos relevantes de cada personaje
        personaje_data = {
            'id': p['url'].split('/')[-2],  # El ID se extrae del enlace
            'name': p['name'],
            'height': p['height'],
            'mass': p['mass'],
            'hair_color': p['hair_color'],
            'skin_color': p['skin_color'],
            'eye_color': p['eye_color'],
            'birth_year': p['birth_year'],
            'gender': p['gender'],
            'homeworld': homeworld,
            'species': ", ".join(especies) if especies else None,
            'films': ", ".join(peliculas)
        }
        data.append(personaje_data)
    
    df = pd.DataFrame(data)
    return df

In [ ]:
# Obtener y mostrar el DataFrame
df_personajes = crear_dataframe()
df_personajes.head()

TypeError: obtener_personajes() missing 2 required positional arguments: 'url' and 'params'

In [2]:
import requests
import pandas as pd
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Función para obtener los datos de los personajes
def obtener_personajes():
    url_base = "https://swapi.dev/api/people/"
    personajes = []
    
    while url_base:
        response = requests.get('https://swapi.dev/api/people/', verify=False)
        data = response.json()
        
        # Añadimos los personajes de la página actual
        personajes.extend(data['results'])
        
        # Si hay una página siguiente, la API la devuelve en la clave 'next'
        url_base = data['next']
    
    return personajes

# Función para obtener los datos adicionales como el planeta, especie y películas
def obtener_datos_adicionales(personaje):
    # Obtener el planeta
    homeworld = requests.get(personaje['homeworld']).json()['name']
    
    # Obtener las especies (si existen)
    especies = []
    if personaje['species']:
        for especie_url in personaje['species']:
            especie = requests.get(especie_url).json()['name']
            especies.append(especie)
    
    # Obtener las películas
    peliculas = []
    for film_url in personaje['films']:
        pelicula = requests.get(film_url).json()['title']
        peliculas.append(pelicula)
    
    return homeworld, especies, peliculas

# Crear un DataFrame de los personajes
def crear_dataframe():
    personajes = obtener_personajes()
    
    data = []
    for p in personajes:
        homeworld, especies, peliculas = obtener_datos_adicionales(p)
        
        # Extraemos los datos relevantes de cada personaje
        personaje_data = {
            'id': p['url'].split('/')[-2],  # El ID se extrae del enlace
            'name': p['name'],
            'height': p['height'],
            'mass': p['mass'],
            'hair_color': p['hair_color'],
            'skin_color': p['skin_color'],
            'eye_color': p['eye_color'],
            'birth_year': p['birth_year'],
            'gender': p['gender'],
            'homeworld': homeworld,
            'species': ", ".join(especies) if especies else None,
            'films': ", ".join(peliculas)
        }
        data.append(personaje_data)
    
    df = pd.DataFrame(data)
    return df

# Obtener y mostrar el DataFrame
df_personajes = crear_dataframe()
df_personajes.head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)